## File Example

In [ ]:
#include "glvis/xglvis.hpp"

In [ ]:
const std::string ex9 = glvis::read_file("/Users/stitt4/workspace/glvis-js/data/ex9.saved");
const std::string ex1 = glvis::read_file("/Users/stitt4/workspace/glvis-js/data/ex1.saved");

In [ ]:
auto glv = glvis::glvis();
glv.show(ex9);
glv

In [ ]:
glv.resize(200, 200)

In [ ]:
glv.show(ex1);

In [ ]:
glv.resize()

## MFEM Example

In [ ]:
# shouldn't be needed?
#pragma cling add_include_path("/Users/stitt4/workspace/mfem-host")
#pragma cling add_library_path("/Users/stitt4/workspace/mfem-host")

In [ ]:
#pragma cling load("libmfem")

In [ ]:
#include <mfem.hpp>
#include <fstream>
#include <iostream>
#include <sstream>

In [ ]:
using namespace std;
using namespace mfem;

Mesh mesh = Mesh::MakeCartesian2D(5, 5, Element::TRIANGLE);
mesh.UniformRefinement();

FiniteElementCollection * fec = new H1_FECollection(2, mesh.Dimension());
FiniteElementSpace fespace(&mesh, fec);
cout << "Number of finite element unknowns: " << fespace.GetTrueVSize() << endl;

Array<int> ess_tdof_list;
if (mesh.bdr_attributes.Size())
{
  Array<int> ess_bdr(mesh.bdr_attributes.Max());
  ess_bdr = 1;
  fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list);
}

LinearForm b(&fespace);
ConstantCoefficient one(1.0);
b.AddDomainIntegrator(new DomainLFIntegrator(one));
b.Assemble();

GridFunction x(&fespace);
x = 0.0;

BilinearForm a(&fespace);
a.AddDomainIntegrator(new DiffusionIntegrator(one));
a.Assemble();

OperatorPtr A;
Vector B, X;
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B);

cout << "Size of linear system: " << A->Height() << endl;

GSSmoother M((SparseMatrix&)(*A));
PCG(*A, M, B, X, 1, 200, 1e-12, 0.0);
a.RecoverFEMSolution(X, b, x);

std::stringstream ss;

ss << "solution\n" << mesh << x << flush;

delete fec;

In [ ]:
auto glv = glvis::glvis();
glv.show(ss.str());
glv